##  <span style='color:green '>Evaluate Targets</span>
 <span style='color:green '>The purpose of this notebook is an attamp to improve upon the previoulsy provided target. The original target was designed to predict a players fantasy value using a target that defines a player by productivity score. This target intentially discards the current fantasy draft rankings in order to identify "sleepers". However this target creates some predictory issues from a human analysis. Two examples that precipitated the reevaluation are Andre Roberts and Jonthan Taylor. These two players represent opposite ends of the prediction spectrum. <span style='color:blue '>Andre Roberts has an original target Value of 288 the highest of all players. His stats are Production of 288, Usage of .1875, 2021 Fantasy points of 7.0 and a current year draft ranking of Null.</span> <span style='color:green '>Modeling projects him to have the highest draft ranking of 288 inline with his production score. However, given that he has no current ADP he should not be considered a viable draft pick. This is cause by having extremely low usage numbers which causes a very high production score. <span style='color:blue '>The other player Jonathan Taylor has an original target value of 97.25, Usage of 3.53, 2021, 2021 Fantasy points of 333 and a current year draft ranking of 1. </span><span style='color:green '>While preserving the scope of the project of finding sleepers, I am attempting to find a new target that will still identify outliers without skewing the results of high ranking high score players. Previous evaluations have demostrated that our ML Models can predict with 99% accuracy the current draft ranking and Production Scores at 98%. This notebook does not intend to improve the modeling but rather the usefulness of the predicted player draft rankings.  </span>
laf 08.27.2022   

In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import math
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler, RobustScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier
from sklearn.model_selection import cross_validate
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score
import joblib
import warnings
warnings.filterwarnings('ignore')
import nfl_data_py as nfl
pd.get_option("display.max_columns")

20

###  <span style='color:green '> Set up variables to be used in Fit Determination Function and Calculations</span>

In [2]:
# Variables Fit Determination Function 
index = []
scores = {'Train Score': [], "Test Score": [], 'Test Train Difference': [], 'Scaler': []}
scalers = ["none", StandardScaler(), MinMaxScaler(), MaxAbsScaler(), RobustScaler()]
models = [LinearRegression(),
          LogisticRegression(),
          KNeighborsRegressor(),
          RandomForestRegressor(),
          RandomForestClassifier(),
          ExtraTreesRegressor(),
          ExtraTreesClassifier(),
          AdaBoostRegressor(),
          AdaBoostClassifier()]

# Variables for year substitution 
C2021 = ['2021 Games', '2021 FantasyPoints','2021 GS','2021 Tgt', '2021 Rec', '2021 RushingTD',
        '2021 RushingYds', '2021 RushingAtt', '2021 ReceivingYds', '2021 ReceivingTD']
C2020 = ['2020 Games', '2020 FantasyPoints','2020 GS','2020 Tgt', '2020 Rec', '2020 RushingTD',
        '2020 RushingYds', '2020 RushingAtt', '2020 ReceivingYds', '2020 ReceivingTD']
C2019 = ['2019 Games', '2019 FantasyPoints','2019 GS','2019 Tgt', '2019 Rec', '2019 RushingTD',
        '2019 RushingYds', '2019 RushingAtt', '2019 ReceivingYds', '2019 ReceivingTD']
CKS = ['Active2021', 'Active2020', 'Active2019']

# Variables for Target Determiniation 
TCC = ['AvgInvert', 'ProdWeighted', 'ProdWeightedSR', 'ProdWeightedSRA' ]

# Labels Columnns to be droped for training 
col = [0,1,2,3]

# Results Column Labels
RSC = ['Player','2019 FantasyPoints', '2020 FantasyPoints', '2021 FantasyPoints','Production21',
       'Average Total Production', '2021 Tm', 'Pos', 'AVG']         

###  <span style='color:green '>Define Function to Test Models and Scalers</span>

In [3]:
def test_model(data):
    for scaler in scalers:
        global index
        global scores
        if scaler != "none":
            scaler.fit(X_train)
            X_train_scaled = scaler.transform(X_train)
            X_test_scaled = scaler.transform(X_test)
        else:
            X_train_scaled = X_train
            X_test_scaled = X_test
        data = X_train_scaled, X_test_scaled, y_train, y_test
        for model in models:
            reg = model.fit(X_train_scaled, y_train)
            y_pred = reg.predict(X_test_scaled)            
            scores["Train Score"].append(reg.score(X_train_scaled, y_train))
            scores["Test Score"].append(reg.score(X_test_scaled, y_test))
            scores["Test Train Difference"].append((reg.score(X_train_scaled, y_train)-(reg.score(X_test_scaled, y_test))))
            scores["Scaler"].append(scaler)
            index += [type(reg).__name__]

###  <span style='color:green '>Get data and evaluate readiness for model</span>

 <span style='color:green '>* Find features and target   
     * Test for imbalance </span>

In [4]:
nfl.see_pbp_cols()
pbp2021 = nfl.import_pbp_data(years=[2021], downcast=True, cache=False, alt_path=None)
pbp2021


# load additional team data 
pbpclean = pbp2021
teams = nfl.import_team_desc()
players = nfl.import_rosters([2021])
player_seasons = nfl.import_seasonal_data([2021])
player_seasons.head(5)

adp = pd.read_csv('Resources/ADPxFinal.csv') 

# When ineligible players are dropped before training the model scores suffer
# due to the limited data available for training. 
# Nulls are preserved and dropped just before the Predicted file is created
# Drop players ineligible to be drafted 
# adp = adp.dropna(subset=['AVG'])

2021 done.
Downcasting floats.


In [5]:
# Preserve label information for Output file 
adp_scope = adp[RSC].copy()

# Match Rows for results set to correction(s) made to training set
# Replace infinite updated data with 'drop'
# drop these rows from what will become our result set 
adp_scope.replace([np.inf, -np.inf], 'drop', inplace=True)
res = adp_scope[~adp_scope.eq('drop').any(1)]
adp_scope  = res
adp_scope

,Player,2019 FantasyPoints,2020 FantasyPoints,2021 FantasyPoints,Production21,Average Total Production,2021 Tm,Pos,AVG
0,Jonathan Taylor,0.0,217.0,333.0,97.250653,59.723157,IND,RB,1.0
1,Austin Ekeler,309.0,111.0,274.0,84.16,79.979791,LAC,RB,3.0
2,Cooper Kupp,270.5,117.0,295.0,100.229167,84.274732,LAR,WR,3.5
3,Derrick Henry,294.6,314.0,175.0,36.853556,67.397175,TEN,RB,3.8
4,Christian McCaffrey,469.2,73.0,91.0,39.35,47.962044,CAR,RB,4.0
...,...,...,...,...,...,...,...,...,...
509,Ethan Wolf,0.0,0.0,1.0,4.0,1.333333,NOR,TE,NaN
510,Nick Boyle,75.1,23.0,0.0,3.333333,35.519423,BAL,TE,NaN
511,Darrell Daniels,1.4,15.0,0.0,0.0,23.245614,ARI,TE,NaN
512,Daniel Helm,0.0,0.0,0.0,-3.0,-1.0,LVR,TE,NaN


In [6]:
# Identify invalid data 
# These columns are drop from the ML Dataset 
# They are added back to the results set they should
# be corrected in the previous step. 
missing_average = adp['AVG'].isna().sum()
missing_pos = adp['Pos'].isna().sum()
missing_team = adp['2021 Tm'].isna().sum()
print(f'Column AVG has {missing_average} null values')
print(f'Column POS has {missing_pos} null values')
print(f'Column Team has {missing_team} null values')

Column AVG has 292 null values
Column POS has 0 null values
Column Team has 0 null values


In [7]:
# Get the mean of the Average of the Average Draft Position 
# These are being used to improve target data for ML Models 
# All AVG Nulls are removed from the prediction dataset 
meanJoeGreen = adp["AVG"].mean()
adp["AVG"] = adp["AVG"].fillna(meanJoeGreen)

In [8]:
# Propgate key statistics for prior years when players were not active in the league. 
# This step is to improve the learning model. This 'dummy data' is dropped after learning/ 

# Create Checksum for years 
adp['Active2021'] = adp[C2021].sum(axis=1)
adp['Active2020'] = adp[C2020].sum(axis=1)
adp['Active2019'] = adp[C2019].sum(axis=1)

# Replace 2020 stats with 2021 stats for learning model 
adp.loc[adp["Active2020"] == 0, '2020 Games'] = adp['2021 Games']
adp.loc[adp["Active2020"] == 0, '2020 FantasyPoints'] = adp['2021 FantasyPoints']
adp.loc[adp["Active2020"] == 0, '2020 GS'] = adp['2021 GS']
adp.loc[adp["Active2020"] == 0, '2020 Tgt'] = adp['2021 Tgt']
adp.loc[adp["Active2020"] == 0, '2020 Rec'] = adp['2021 Rec']
adp.loc[adp["Active2020"] == 0, '2020 RushingYds'] = adp['2021 RushingYds']
adp.loc[adp["Active2020"] == 0, '2020 RushingAtt'] = adp['2021 RushingAtt']
adp.loc[adp["Active2020"] == 0, '2020 ReceivingYds'] = adp['2021 ReceivingYds']
adp.loc[adp["Active2020"] == 0, '2020 ReceivingTD'] = adp['2021 ReceivingTD']

# Replace 2019 stats with 2020 stats for learning model 
adp.loc[adp["Active2019"] == 0, '2019 Games'] = adp['2020 Games']
adp.loc[adp["Active2019"] == 0, '2019 FantasyPoints'] = adp['2020 FantasyPoints']
adp.loc[adp["Active2019"] == 0, '2019 GS'] = adp['2020 GS']
adp.loc[adp["Active2019"] == 0, '2019 Tgt'] = adp['2020 Tgt']
adp.loc[adp["Active2019"] == 0, '2019 Rec'] = adp['2020 Rec']
adp.loc[adp["Active2019"] == 0, '2019 RushingTD'] = adp['2020 RushingTD']
adp.loc[adp["Active2019"] == 0, '2019 RushingAtt'] = adp['2020 RushingAtt']
adp.loc[adp["Active2019"] == 0, '2019 ReceivingYds'] = adp['2020 ReceivingYds']
adp.loc[adp["Active2019"] == 0, '2019 ReceivingTD'] = adp['2020 ReceivingTD']


# Verify all 2021 players are active 
notA21 = list(adp['Active2021']).count(0)
notA20 = list(adp['Active2020']).count(0)
notA19 = list(adp['Active2019']).count(0)

print(f'Not Active in 2019: {notA19} Not Active in 2020: {notA20} Not Active in 2021: {notA21}')

# showme = adp[adp['Player'] == 'Najee Harris'] 
# pd.set_option("display.max_columns", None)
# showme

Not Active in 2019: 191 Not Active in 2020: 120 Not Active in 2021: 0


In [9]:
# Attempt to improve target (see notebook title cell for additional details)
# Reduce the error rate created in production created by players who played a small number of games 
# Reduces the "hope factor" of players injured in the 2021 season who are currently drafting high based on Pre-season
# reassigning the above players score adjusted by actual performances 
# Correct the number being skewed by the number of games played in 2021(when active players in 2021)

# Invert ADP
adp['AvgInvert'] = (adp['AVG'].max() + 1) - adp['AVG']
# Give Production Scores and Most recent Fantasy Point equal Weight
adp['ProdWeighted'] = adp['Production21'] * adp['2021 FantasyPoints']
# Take the square root of the wieghted results
adp['ProdWeightedSR'] = adp['ProdWeighted']**(1/2)
# Factor in "human factor" of current ADP
adp['ProdWeightedSRA'] = (adp['ProdWeightedSR'] + adp['AvgInvert'])/2
# Correct any Nulls created by source data errors 
adp["ProdWeightedSRA"] = adp["ProdWeightedSRA"].fillna(114)

# Display for analysis 
# adp.sort_values('ProdWeighted', ascending=False).head(50)
# adp.sort_values('AVG', ascending=True).head(50)

In [10]:
# Create df with calculated data
adp_production = adp[['Player',
                     '2021 FantasyPoints',
                     'AVG',
                     'ProdWeightedSRA', 
                     'Production21',
                     'ProdWeighted',
                     'ProdWeightedSR',
                     'AvgInvert',
                     '2021 Games',
                    ]].copy()

adp_by_weight = adp_production.sort_values('ProdWeightedSRA', ascending=False).reset_index()

In [11]:
# Error rate represents the change in value between our new target of ProdWeightedSRA(ranked)
# and the current average ADP as reported by Fantasy Draft sites. 
adp_by_weight['Error Rate'] = (adp_by_weight.index + 1) - adp_by_weight['AVG']
# adp_by_weight.head(5)

In [12]:
# Single player display all columns for analysis 
# showme = adp[adp['Player'] == 'Andre Roberts'] 
# pd.set_option("display.max_columns", None)
# showme = adp[adp['Player'] == 'Jonathan Taylor'] 
# pd.set_option("display.max_columns", None)
# showme

In [13]:
# Check for remaining nulls 
count_nan = adp.isna().sum().sum()
if count_nan > 0:
    null_rows = adp[adp.isna().any(axis=1)]
    null_rows

In [14]:
# Replace infinite updated data with nan in training data 
adp.replace([np.inf, -np.inf], 'drop', inplace=True)

# Drop rows with 'drop' that were just created by the infinity value replacements
res = adp[~adp.eq('drop').any(1)]
adp  = res

In [15]:
# Save Potential targets 
# Set Target Variable 
Original_Target = adp["Production21"].values
AltTarget = adp["AVG"].values
AltTarget1 = adp["ProdWeightedSRA"].values

# Set Target Variable 
Target = AltTarget1 

In [16]:
# Remove non-target columns
# Remove unnamed column, Player, Pos and 2021 Team
adp.drop(adp.columns[col],axis=1,inplace=True)
# Drop Check Sum columns from DataFrame 
adp.drop(columns=CKS, inplace=True)
# Drop Target Calc columns from DataFrame 
adp.drop(columns=TCC, inplace=True)

In [17]:
# Look for non numerics 
adp.applymap(np.isreal)
invalidNumbers = adp[~adp.applymap(np.isreal).all(1)]
if len(invalidNumbers) > 0:
    print((f'There are {len(invalidNumbers)} rows with invaid numeric data'))

In [18]:
print(len(Target))
adp

510


,Age,2021 Games,2021 GS,2021 Tgt,2021 Rec,2021 RushingYds,2021 RushingTD,2021 RushingAtt,2021 ReceivingYds,2021 ReceivingTD,...,TotYdsScrm20,Production20,Usage21,Touchdowns21,TotYdsScrm21,Production21,Average Total Usage,Average Total Yards,Average Total Production,AVG
0,22.0,17,17,51.0,40.0,1811.0,18.0,332.0,360.0,2.0,...,1468.0,81.918819,22.529412,20.0,2171.0,97.250653,13.532026,1213.000000,59.723157,1.000000
1,26.0,16,16,94.0,70.0,911.0,12.0,206.0,647.0,8.0,...,933.0,51.712707,18.750000,20.0,1558.0,84.16,17.283333,1347.000000,79.979791,3.000000
2,28.0,17,17,191.0,145.0,18.0,0.0,4.0,1947.0,16.0,...,1007.0,70.138889,19.764706,16.0,1965.0,100.229167,16.138235,1379.000000,84.274732,3.500000
3,27.0,8,8,20.0,18.0,937.0,10.0,219.0,154.0,0.0,...,2141.0,84.420538,29.875000,10.0,1091.0,36.853556,25.745833,1659.333333,67.397175,3.800000
4,25.0,7,7,41.0,37.0,442.0,1.0,99.0,343.0,1.0,...,374.0,14.615385,20.000000,2.0,785.0,39.35,24.270833,1183.666667,47.962044,4.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
509,26.0,2,1,2.0,2.0,0.0,0.0,0.0,8.0,0.0,...,0.0,0.0,2.000000,0.0,8.0,4.0,0.666667,2.666667,1.333333,181.646847
510,28.0,5,3,2.0,1.0,0.0,0.0,0.0,2.0,0.0,...,113.0,33.387097,0.600000,0.0,2.0,3.333333,2.889815,145.333333,35.519423,181.646847
511,27.0,15,4,4.0,1.0,0.0,0.0,0.0,0.0,0.0,...,92.0,58.736842,0.333333,0.0,0.0,0.0,0.760101,32.000000,23.245614,181.646847
512,26.0,9,1,2.0,1.0,0.0,0.0,0.0,-1.0,0.0,...,0.0,0.0,0.333333,0.0,-1.0,-3.0,0.111111,-0.333333,-1.0,181.646847


###  <span style='color:green '>Apply Scaling for Initial Model Test </span>

In [19]:
# Standarize data with StandardScaler
# adps = StandardScaler().fit_transform(adp)
# When best Model changed - the best scaler changed
adps = MinMaxScaler().fit_transform(adp)
print(adps[0:1])

[[0.0625     1.         1.         0.26701571 0.27586207 1.
  1.         1.         0.18615385 0.125      1.         0.875
  0.76470588 0.24840764 0.31034483 0.00773196 0.6875     0.75570033
  0.20604396 0.07692308 0.4647708  0.875      0.76470588 0.23493976
  0.28346457 0.57920549 0.64705882 0.61375661 0.19635891 0.05555556
  0.69303797 0.         0.         0.         0.         0.69107468
  0.57142857 0.68566091 0.48761202 0.75363748 1.         1.
  0.35774631 0.5246538  0.7310705  0.48222749 0.        ]]


###  <span style='color:green '> PCA</span>
<span style='color:green '> Applying PCA to reduce dimensions while preserving 99% of the explained variance </span>

In [20]:
# Applying PCA to reduce dimensions while preserving 90% of the explained variance 
# Initialize PCA model by setting n-components to desired level
pca = PCA(n_components= .99)

# Fit our new Principal Component Analysis reduced Features to our Model
pfa = pca.fit_transform(adps)

# Transform PCA data to a DataFrame
pf = pd.DataFrame(data=pfa)
pf.head(2)

,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,1.860503,1.590181,1.194111,1.078909,-0.260434,0.287427,0.054412,0.526173,0.166212,-0.026854,...,0.165960,-0.073855,0.205012,-0.047363,-0.165789,-0.022301,0.169774,0.309970,-0.231826,0.079390
1,2.236539,0.865727,0.056710,0.422581,0.652730,-0.319442,-0.399654,0.176146,-0.025384,0.300054,...,0.358934,-0.209976,0.473594,0.117136,0.114263,0.130214,0.057643,-0.024516,-0.064482,0.015488


###  <span style='color:green '>Create X and Y | Reshape the data</span>

In [21]:
# Create X (features) and y (target) sets
X = pf
y = Target

print("Shape: ", X.shape, y.shape)

Shape:  (510, 23) (510,)


###  <span style='color:green '>Split data into Training and Testing sets</span>

In [22]:
# Split the data into training and testing sets
data = X_train, X_test, y_train, y_test = train_test_split(X, y, random_state= 42)

###  <span style='color:green '>Get to know our data</span>

In [23]:
%%time
# setting Random forest as sample to view data 
# classifier = ExtraTreesRegressor()
classifier = LinearRegression()
clf = classifier.fit(X_train, y_train)
print(f"Training Data Score: {classifier.score(X_train, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test, y_test)}")

# Does not apply to linear Regression 
# features = clf.feature_importances_
# print(features)
# plt.bar(x = range(len(features)), height=features)
# plt.show()

Training Data Score: 0.9960708848263538
Testing Data Score: 0.9954345688673915
Wall time: 24.3 ms


In [24]:
# Does not apply to linear Regression 
# %%time
# features = sorted(zip(X.columns, clf.feature_importances_), key = lambda x: x[1])
# cols = [f[0] for f in features]
# width = [f[1] for f in features]

# fig, ax = plt.subplots()
# fig.set_size_inches(10,5)
# plt.margins(y=0.001)

# ax.barh(y=cols, width=width)

# plt.show()

In [25]:
# Does not apply to linear Regression 
# %%time
# features = sorted(zip(X.columns, clf.feature_importances_), key = lambda x: x[1])
# features = features[:10]
# cols = [f[0] for f in features]
# width = [f[1] for f in features]

# fig, ax = plt.subplots()
# fig.set_size_inches(10,5)
# plt.margins(y=0.001)

# ax.barh(y=cols, width=width)

# plt.show()

In [26]:
y_predicted = clf.predict(X_test)
scores1 = clf.score(X_test, y_test)

print(y_predicted)
print(scores1)

[119.48607812 124.18181755 123.63063433 136.17569596 118.58396807
 219.45042858 122.26829728 183.17765325 121.57484743 135.92670141
 112.87578206  17.18095407 138.64359827 128.58141729  78.23804927
 165.70683189 113.77795058 123.27925259 112.21890945 206.48174829
 112.48103455 274.1604282  132.14417065 123.15075621 113.6034328
 120.93112175  87.72685951 186.31140799 132.12302538 134.4892934
 121.57705267 128.89543043 130.32130014 188.18683627  58.60662426
 112.09215575 123.26006144 148.26849833  11.31140818  89.05975674
 154.70014878 114.93373692 222.59072177 301.55644949 135.18325909
 114.75058918 186.46146795 123.87065262  58.23716292 120.2817134
  81.42236111 189.4356834  120.04786944 127.9112462  118.74732094
 111.2851463  243.2026517  195.19343114 123.20113153  84.82712009
 221.52955699 299.05159657 183.01174552 263.74712795 143.93484465
 251.9833096  113.76967756 189.15366687 134.4162108  196.82186977
 116.88399723 116.90355323 248.29342382 121.32039087 134.32228069
 132.15914015

In [27]:
y_predicted = clf.predict(X)
print(y_predicted)

[299.05159657 290.08181737 301.55644949 247.27028527 233.06302978
 282.06061096 268.14238128 249.73919409 278.8122707  274.1604282
 271.88040407 263.74712795 254.90999799 257.04062015 245.22324973
 250.8805018  253.26164261 285.18675543 248.29342382 247.31341342
 230.67124617 257.08315414 251.9833096  260.48881297 255.36253192
 232.85189265 244.26619057 251.73182707 242.50508072 252.77876101
 186.31140799 234.34997829 242.3210146  243.2026517  235.10322706
 235.31039384 231.0930886  242.63707207 235.70155653 222.61647206
 210.13766157 241.26440556 226.62134784 213.3959278  194.72080116
 232.16192702 225.18131751 221.99733418 204.07669203 220.31346795
 201.18982827 221.43330061 225.78593423 201.38232354 206.50462963
 219.45042858 210.39486966 219.89601053 169.28968723 206.83567659
 189.29057284 216.29456596 198.53688109 222.59072177 205.40865017
 195.58128468 196.95579719 185.49745333 221.52955699 194.87537215
 206.48174829 200.9251389  196.82186977 188.18683627 215.79560653
 202.543248

In [28]:
adp_scope['Prediction'] = y_predicted
adp_scope.rename(columns={"2021 Tm": "Team"})

,Player,2019 FantasyPoints,2020 FantasyPoints,2021 FantasyPoints,Production21,Average Total Production,Team,Pos,AVG,Prediction
0,Jonathan Taylor,0.0,217.0,333.0,97.250653,59.723157,IND,RB,1.0,299.051597
1,Austin Ekeler,309.0,111.0,274.0,84.16,79.979791,LAC,RB,3.0,290.081817
2,Cooper Kupp,270.5,117.0,295.0,100.229167,84.274732,LAR,WR,3.5,301.556449
3,Derrick Henry,294.6,314.0,175.0,36.853556,67.397175,TEN,RB,3.8,247.270285
4,Christian McCaffrey,469.2,73.0,91.0,39.35,47.962044,CAR,RB,4.0,233.063030
...,...,...,...,...,...,...,...,...,...,...
509,Ethan Wolf,0.0,0.0,1.0,4.0,1.333333,NOR,TE,NaN,112.481035
510,Nick Boyle,75.1,23.0,0.0,3.333333,35.519423,BAL,TE,NaN,111.216093
511,Darrell Daniels,1.4,15.0,0.0,0.0,23.245614,ARI,TE,NaN,113.462304
512,Daniel Helm,0.0,0.0,0.0,-3.0,-1.0,LVR,TE,NaN,112.523754


In [29]:
adp_scope.sort_values('Prediction', ascending=False).head(50)

,Player,2019 FantasyPoints,2020 FantasyPoints,2021 FantasyPoints,Production21,Average Total Production,2021 Tm,Pos,AVG,Prediction
2,Cooper Kupp,270.50,117.0,295.0,100.229167,84.274732,LAR,WR,3.5,301.556449
0,Jonathan Taylor,0.00,217.0,333.0,97.250653,59.723157,IND,RB,1.0,299.051597
1,Austin Ekeler,309.00,111.0,274.0,84.16,79.979791,LAC,RB,3.0,290.081817
17,Deebo Samuel,187.10,48.0,262.0,144.161616,95.756771,SFO,WR,19.0,285.186755
5,Justin Jefferson,0.00,186.0,222.0,101.381818,69.074065,MIN,WR,6.3,282.060611
8,Ja'Marr Chase,0.00,0.0,224.0,121.114833,40.371611,CIN,WR,9.3,278.812271
9,Joe Mixon,225.40,79.0,246.0,72.235294,55.585495,CIN,RB,9.8,274.160428
10,Davante Adams,210.70,243.0,221.0,85.69863,72.257985,GNB,WR,11.5,271.880404
6,Najee Harris,0.00,0.0,227.0,71.094763,23.698254,PIT,RB,7.3,268.142381
11,Stefon Diggs,212.10,202.0,183.0,78.632959,92.43667,BUF,WR,12.8,263.747128


In [30]:
adp_scope.sort_values('AVG', ascending=True).head(50)

,Player,2019 FantasyPoints,2020 FantasyPoints,2021 FantasyPoints,Production21,Average Total Production,2021 Tm,Pos,AVG,Prediction
0,Jonathan Taylor,0.00,217.0,333.0,97.250653,59.723157,IND,RB,1.0,299.051597
1,Austin Ekeler,309.00,111.0,274.0,84.16,79.979791,LAC,RB,3.0,290.081817
2,Cooper Kupp,270.50,117.0,295.0,100.229167,84.274732,LAR,WR,3.5,301.556449
3,Derrick Henry,294.60,314.0,175.0,36.853556,67.397175,TEN,RB,3.8,247.270285
4,Christian McCaffrey,469.20,73.0,91.0,39.35,47.962044,CAR,RB,4.0,233.063030
5,Justin Jefferson,0.00,186.0,222.0,101.381818,69.074065,MIN,WR,6.3,282.060611
6,Najee Harris,0.00,0.0,227.0,71.094763,23.698254,PIT,RB,7.3,268.142381
7,Dalvin Cook,292.40,294.0,172.0,60.59396,69.724218,MIN,RB,7.8,249.739194
8,Ja'Marr Chase,0.00,0.0,224.0,121.114833,40.371611,CIN,WR,9.3,278.812271
9,Joe Mixon,225.40,79.0,246.0,72.235294,55.585495,CIN,RB,9.8,274.160428


In [31]:
# save
joblib.dump(classifier, "Resources/draft_position_no_QB.joblib")

# Temporary write file while we are working on data ***
adp_scope = adp_scope.dropna(subset=['AVG'])

adp_scope.to_csv('Resources/Draft_position_no_QB.csv', index=False)

adp_scope

,Player,2019 FantasyPoints,2020 FantasyPoints,2021 FantasyPoints,Production21,Average Total Production,2021 Tm,Pos,AVG,Prediction
0,Jonathan Taylor,0.0,217.0,333.0,97.250653,59.723157,IND,RB,1.0,299.051597
1,Austin Ekeler,309.0,111.0,274.0,84.16,79.979791,LAC,RB,3.0,290.081817
2,Cooper Kupp,270.5,117.0,295.0,100.229167,84.274732,LAR,WR,3.5,301.556449
3,Derrick Henry,294.6,314.0,175.0,36.853556,67.397175,TEN,RB,3.8,247.270285
4,Christian McCaffrey,469.2,73.0,91.0,39.35,47.962044,CAR,RB,4.0,233.063030
...,...,...,...,...,...,...,...,...,...,...
217,Benny Snell Jr.,0.0,65.0,11.0,47.175,34.199667,PIT,RB,394.0,21.497482
218,Anthony McFarland Jr.,0.0,17.0,1.0,7.0,16.912698,PIT,RB,395.0,11.311408
219,Equanimeous St. Brown,0.0,18.0,11.0,56.0,43.666667,GNB,WR,397.0,21.243973
220,Kene Nwangwu,0.0,0.0,19.0,42.777778,14.259259,MIN,RB,399.0,17.180954


### <span style='color:green '>The section below is to evaluate new potential Models </span>

In [32]:
# Create X (features) and y (target) sets
X = pf

#convert y values to categorical values(for model selection matrix)
lab = preprocessing.LabelEncoder()
y_transformed = lab.fit_transform(y)
y = y_transformed

# Split the data into training and testing sets
data = X_train, X_test, y_train, y_test = train_test_split(X, y, random_state= 42)
print(len(y))
print(len(X))

510
510


In [33]:
%%time
# call function test_model to test all models being considered
scores = {'Train Score': [], "Test Score": [], 'Test Train Difference': [], 'Scaler': []}
test_model(data)

Wall time: 49 s


In [34]:
# Print by Training Score
df_scores = pd.DataFrame(scores, index=index)
df_scores.sort_values('Train Score', ascending=False).head(10)

,Train Score,Test Score,Test Train Difference,Scaler
RandomForestClassifier,1.0,0.000000,1.000000,MinMaxScaler()
ExtraTreesClassifier,1.0,0.000000,1.000000,MinMaxScaler()
RandomForestClassifier,1.0,0.007812,0.992188,MaxAbsScaler()
ExtraTreesRegressor,1.0,0.783223,0.216777,MaxAbsScaler()
ExtraTreesClassifier,1.0,0.000000,1.000000,MaxAbsScaler()
ExtraTreesClassifier,1.0,0.000000,1.000000,StandardScaler()
ExtraTreesRegressor,1.0,0.780156,0.219844,StandardScaler()
RandomForestClassifier,1.0,0.000000,1.000000,StandardScaler()
ExtraTreesRegressor,1.0,0.789277,0.210723,MinMaxScaler()
RandomForestClassifier,1.0,0.007812,0.992188,RobustScaler()


In [35]:
# Print by Testing Score
df_scores.sort_values('Test Score', ascending=False).head(10)

,Train Score,Test Score,Test Train Difference,Scaler
LinearRegression,0.840599,0.808993,0.031606,none
LinearRegression,0.840599,0.808993,0.031606,StandardScaler()
LinearRegression,0.840599,0.808993,0.031606,MaxAbsScaler()
LinearRegression,0.840599,0.808993,0.031606,MinMaxScaler()
LinearRegression,0.840599,0.808993,0.031606,RobustScaler()
ExtraTreesRegressor,1.000000,0.790284,0.209716,RobustScaler()
ExtraTreesRegressor,1.000000,0.789277,0.210723,MinMaxScaler()
ExtraTreesRegressor,1.000000,0.786264,0.213736,none
ExtraTreesRegressor,1.000000,0.783223,0.216777,MaxAbsScaler()
ExtraTreesRegressor,1.000000,0.780156,0.219844,StandardScaler()


In [36]:
# Print by Testing Score
df_scores['Test Train Magnitude'] = abs(df_scores['Test Train Difference'])
df_scores.sort_values('Test Train Magnitude').head(10)

,Train Score,Test Score,Test Train Difference,Scaler,Test Train Magnitude
AdaBoostClassifier,0.034031,0.007812,0.026219,MinMaxScaler(),0.026219
LogisticRegression,0.036649,0.007812,0.028837,MinMaxScaler(),0.028837
AdaBoostClassifier,0.036649,0.007812,0.028837,none,0.028837
AdaBoostClassifier,0.039267,0.007812,0.031455,RobustScaler(),0.031455
LinearRegression,0.840599,0.808993,0.031606,StandardScaler(),0.031606
LinearRegression,0.840599,0.808993,0.031606,none,0.031606
LinearRegression,0.840599,0.808993,0.031606,RobustScaler(),0.031606
LinearRegression,0.840599,0.808993,0.031606,MaxAbsScaler(),0.031606
LinearRegression,0.840599,0.808993,0.031606,MinMaxScaler(),0.031606
AdaBoostClassifier,0.041885,0.007812,0.034072,MaxAbsScaler(),0.034072


#### The above model testing results are only directionally predictive of fit for some models. This is because the target has been label encoded for all of the models but is only required for some of the models. You can see by the model creation step earlier in the notebook that the actual results for liner regression model has a training score of .996 and a testing score of .995
